In [1]:
import pandas as pd
from pandas import DataFrame, Series
df_train = pd.read_csv('100 data/ratings_test.txt', delimiter='\t', keep_default_na=False)

In [2]:
df_train['document'][:10]

0                                                  굳 ㅋ
1                                 GDNTOPCLASSINTHECLUB
2               뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                     지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4    3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
5                                   음악이 주가 된, 최고의 음악영화
6                                              진정한 쓰레기
7             마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
8    갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...
9       이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..
Name: document, dtype: object

In [3]:
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
from konlpy.tag import Okt
twitter_tag = Okt()

In [5]:
# 형태소 분석 방법(조사, 어미 등 제외)
def twitter_tokenizer_part(text):
    lst = []
    for tpl in twitter_tag.pos(text, stem=True):
        if not tpl[1] in ["Josa", "Eomi", "PreEomi", "Punctuation"]:
            lst.append(tpl[0])
    if len(lst) != 0: 
        return lst
    else:
        return ["뷁"]

In [6]:
text = df_train['document'][:100]
vect = CountVectorizer(tokenizer=twitter_tokenizer_part).fit(text)

In [7]:
print(vect.get_feature_names()[80:90])
print(len(vect.get_feature_names()))

['괜히', '굉장하다', '교훈', '구성', '군', '굳다', '굿굿', '궁금', '귀엽다', '그']
622


In [8]:
tokenized_words = []
for line in text:
    for tpl in twitter_tag.pos(line, stem=True):
        if not tpl[1] in ["Josa", "Eomi", "PreEomi", "Punctuation"]:
            tokenized_words.append(tpl[0])

In [9]:
print(tokenized_words[10:20])
print(len(tokenized_words))

['점', '짜다', '리', '더', '더욱', '아니다', '지루하다', '않다', '완전', '막장']
981


In [12]:
x_train = []
y_train = []
for i in range(len(tokenized_words)-3):
    x_train.append(vect.transform([tokenized_words[i],tokenized_words[i+1],tokenized_words[i+2]]).toarray())
    y_train.append(vect.transform([tokenized_words[i+3]]).toarray())

In [13]:
y_train[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
y_train[0].shape

(1, 622)

In [15]:
len(x_train)

978

In [16]:
# 각 단어 representation을 622차원에서 2차원으로 축소
X = tf.placeholder(tf.float32, [3,622]) #x_train의 각 값을 넣을 것임 
W = tf.Variable(tf.random_normal([622, 2], seed=0))
b = tf.Variable(tf.random_normal([3,2], seed=0))
expr = tf.matmul(X, W) + b

In [17]:
x_data = []
for i in range(len(x_train)):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    #print("=== W ===")
    #print(sess.run(W))
    #print("=== b ===")
    #print(sess.run(b))
    x_data.append(sess.run(expr, feed_dict={X: x_train[i]}).flatten())

print(x_data[0])

[-0.55631971  4.9211874   1.29128885  0.70431834 -3.6267972  -0.35812882]


In [18]:
from keras.models import Model
from keras.layers import Dense, Input

Using TensorFlow backend.


In [29]:
x = Input([6,]) # input node는 2 * 3 = 6임
h = Dense(128, activation='tanh')(x) # hidden node는 임의의 2의 승수 값으로 지정
y = Dense(622, activation='softmax')(h) # output node는 one-hot encoding 차원인 640
model = Model(x,y)
model.compile(loss="categorical_crossentropy", optimizer='sgd', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               896       
_________________________________________________________________
dense_6 (Dense)              (None, 622)               80238     
Total params: 81,134
Trainable params: 81,134
Non-trainable params: 0
_________________________________________________________________


In [30]:
# data를 array 형태로 변형
import numpy as np
x_data = np.array(x_data).reshape(len(x_data),6)
y_train = np.array(y_train).reshape(len(y_train),622)

In [31]:
print(x_data.shape)
print(y_train.shape)

(978, 6)
(978, 622)


In [32]:
model.fit(x_data, y_train, epochs=1000)

Epoch 1/1000
978/978 [==============================] - 1s 775us/step - loss: 6.3580 - acc: 0.0000e+00
Epoch 2/1000
978/978 [==============================] - 0s 421us/step - loss: 6.3373 - acc: 0.0000e+00
Epoch 3/1000
978/978 [==============================] - 0s 465us/step - loss: 6.3176 - acc: 0.0010
Epoch 4/1000
978/978 [==============================] - 0s 435us/step - loss: 6.2979 - acc: 0.0102
Epoch 5/1000
978/978 [==============================] - 0s 501us/step - loss: 6.2788 - acc: 0.0123
Epoch 6/1000
978/978 [==============================] - 0s 464us/step - loss: 6.2604 - acc: 0.0215
Epoch 7/1000
978/978 [==============================] - 0s 481us/step - loss: 6.2426 - acc: 0.0256
Epoch 8/1000
978/978 [==============================] - 0s 503us/step - loss: 6.2250 - acc: 0.0235
Epoch 9/1000
978/978 [==============================] - 0s 508us/step - loss: 6.2079 - acc: 0.0245
Epoch 10/1000
978/978 [==============================] - 0s 479us/step - loss: 6.1916 - acc: 0.0225
E

Epoch 967/1000
978/978 [==============================] - 0s 435us/step - loss: 3.1820 - acc: 0.3742
Epoch 968/1000
978/978 [==============================] - 0s 426us/step - loss: 3.1797 - acc: 0.3783
Epoch 969/1000
978/978 [==============================] - 0s 458us/step - loss: 3.1775 - acc: 0.3793
Epoch 970/1000
978/978 [==============================] - 0s 429us/step - loss: 3.1754 - acc: 0.3804
Epoch 971/1000
978/978 [==============================] - 0s 435us/step - loss: 3.1734 - acc: 0.3814
Epoch 972/1000
978/978 [==============================] - 0s 432us/step - loss: 3.1711 - acc: 0.3824
Epoch 973/1000
978/978 [==============================] - 0s 431us/step - loss: 3.1690 - acc: 0.3783
Epoch 974/1000
978/978 [==============================] - 0s 436us/step - loss: 3.1669 - acc: 0.3793
Epoch 975/1000
978/978 [==============================] - 0s 424us/step - loss: 3.1647 - acc: 0.3855
Epoch 976/1000
978/978 [==============================] - 0s 437us/step - loss: 3.1625 - ac

In [33]:
pre = model.predict(x_data)

In [34]:
for i in range(30):
    print(tokenized_words[i: i+3], end='===>')
    print(pre[i].argmax(), end=' ')
    print(vect.get_feature_names()[pre[i].argmax()])

['굳다', 'ㅋ', 'GDNTOPCLASSINTHECLUB']===>252 뭐
['ㅋ', 'GDNTOPCLASSINTHECLUB', '뭐']===>445 음악
['GDNTOPCLASSINTHECLUB', '뭐', '이']===>595 하다
['뭐', '이', '평점']===>185 들다
['이', '평점', '들']===>178 되다
['평점', '들', '나쁘다']===>67 결국
['들', '나쁘다', '않다']===>1 10
['나쁘다', '않다', '10']===>496 점
['않다', '10', '점']===>539 짜다
['10', '점', '짜다']===>493 적
['점', '짜다', '리']===>237 몇번
['짜다', '리', '더']===>166 더욱
['리', '더', '더욱']===>219 막판
['더', '더욱', '아니다']===>528 지루하다
['더욱', '아니다', '지루하다']===>576 톰
['아니다', '지루하다', '않다']===>576 톰
['지루하다', '않다', '완전']===>511 좀
['않다', '완전', '막장']===>595 하다
['완전', '막장', '임']===>450 이
['막장', '임', '돈']===>515 주다
['임', '돈', '주다']===>450 이
['돈', '주다', '보기']===>10 3
['주다', '보기', '3']===>20 d
['보기', '3', 'D']===>220 만
['3', 'D', '만']===>512 좋다
['D', '만', '아니다']===>294 비교
['만', '아니다', '별']===>185 들다
['아니다', '별', '다섯']===>143 놓다
['별', '다섯', '개']===>416 영화
['다섯', '개', '주다']===>450 이
